#  AM Check-in Data Prep

In [8]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.0.0) is available. Your version (1.9.4) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Sean Deering!



### Get Raw AM Check-in Data

In [9]:
amcheckin_raw_id = 'syn7115672'
amcheckin = pd.read_csv(syn.get(amcheckin_raw_id).path)

### Process AM Check-in Data

In [10]:
#convert seconds to minutes    
amcheckin['AMCH-2a'] = amcheckin['AMCH-2a']/60
amcheckin['AMCH-3a'] = amcheckin['AMCH-3a']/60
amcheckin['AMCH-5'] = amcheckin['AMCH-5']/60

#renaming these so they are consistent with PM Check-in
amcheckin = amcheckin.rename(columns = 
                             { 'AMCH-1':'AMCH1',
                              'AMCH-2':'AMCH2',
                              'AMCH-2a':'AMCH2A',
                              'AMCH-3':'AMCH3',
                              'AMCH-3a':'AMCH3A',
                              'AMCH-4':'AMCH4',
                              'AMCH-5':'AMCH5'})

### Filter based on age

In [11]:
underage_participants = pd.read_csv( syn.get('syn21905452').path, sep="\t")
amcheckin = amcheckin[~amcheckin.participantId.isin(underage_participants.participantId)]

### Remove test accounts

In [12]:
test_accounts = pd.read_excel(syn.get('syn21958537').path)

amcheckin = amcheckin[~amcheckin.participantId.isin(test_accounts.participantId)]

### Replace white space with NaN

In [13]:
amcheckin = amcheckin.replace(r'^\s*$', np.nan, regex=True)

### Clean  AM-Checkin Data (Thresholds for outliers commented out, all data provided).

### AMCH2A Cleaning (Sleep Onset Latency)

In [14]:
amcheckin.AMCH2A.describe()

count    14544.000000
mean        67.704277
std         87.526330
min          1.000000
25%         25.000000
50%         45.000000
75%         70.000000
max       1439.000000
Name: AMCH2A, dtype: float64

In [15]:
amcheckin.AMCH2A.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

0.010       1.00
0.050      10.00
0.500      45.00
0.600      60.00
0.800      90.00
0.850     120.00
0.900     121.00
0.950     200.00
0.970     244.42
0.990     409.13
0.999    1343.85
1.000    1439.00
Name: AMCH2A, dtype: float64

In [16]:
#exclude data above 99th percentile, which in this case is 409.13 minutes
#amcheckin.AMCH2A[ amcheckin.AMCH2A > 409.13 ] = 'LOW DATA QUALITY'


In [17]:
amcheckin.AMCH2A.value_counts()

30.0     2107
60.0     1898
20.0     1147
61.0      981
15.0      868
45.0      741
120.0     735
90.0      502
10.0      497
121.0     466
40.0      358
1.0       345
25.0      334
180.0     259
181.0     192
35.0      167
240.0     137
150.0     120
75.0      120
5.0       110
50.0       92
241.0      76
80.0       69
300.0      65
210.0      64
105.0      60
16.0       58
17.0       50
12.0       49
70.0       48
         ... 
164.0       1
184.0       1
161.0       1
154.0       1
147.0       1
469.0       1
299.0       1
97.0        1
413.0       1
221.0       1
179.0       1
269.0       1
965.0       1
117.0       1
101.0       1
472.0       1
256.0       1
720.0       1
144.0       1
138.0       1
476.0       1
178.0       1
298.0       1
201.0       1
129.0       1
900.0       1
275.0       1
118.0       1
503.0       1
149.0       1
Name: AMCH2A, Length: 259, dtype: int64

### AMCH 3 Cleaning (# of wakes during the night)

In [18]:
amcheckin.AMCH3.describe()

count      72609.000000
mean          17.182250
std         3729.628531
min            0.000000
25%            0.000000
50%            1.000000
75%            3.000000
max      1000000.000000
Name: AMCH3, dtype: float64

In [19]:
amcheckin.AMCH3.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

0.010          0.0
0.050          0.0
0.500          1.0
0.600          2.0
0.800          3.0
0.850          3.0
0.900          4.0
0.950          5.0
0.970          7.0
0.990         10.0
0.999         25.0
1.000    1000000.0
Name: AMCH3, dtype: float64

In [20]:
#exclude anything above 1400 wakes
#amcheckin.AMCH3[ amcheckin.AMCH3 > 1400 ] = 'LOW DATA QUALITY'

In [21]:
amcheckin.AMCH3.value_counts()

1.0          19230
0.0          18713
2.0          15007
3.0           9328
4.0           4088
5.0           2749
6.0           1149
8.0            572
7.0            481
10.0           443
9.0            202
12.0           124
15.0           104
11.0            94
13.0            53
20.0            51
14.0            49
16.0            31
17.0            19
18.0            15
19.0            14
25.0            14
30.0            11
22.0             9
21.0             8
28.0             6
100.0            3
26.0             3
50.0             2
34.0             2
31.0             2
615.0            2
35.0             2
400.0            2
43.0             2
555.0            2
32.0             1
99.0             1
52.0             1
82.0             1
55.0             1
100000.0         1
800.0            1
39.0             1
68.0             1
24.0             1
1122.0           1
87.0             1
885.0            1
96.0             1
90.0             1
36.0             1
1000000.0   

### AMCH3A Data Cleaning (Total Time Awake Overnight)

In [22]:
amcheckin.AMCH3A.describe()

count    54065.000000
mean        43.547785
std         98.876173
min          1.000000
25%          5.000000
50%         15.000000
75%         45.000000
max       1439.000000
Name: AMCH3A, dtype: float64

In [23]:
amcheckin.AMCH3A.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

0.010       1.0
0.050       1.0
0.500      15.0
0.600      20.0
0.800      60.0
0.850      61.0
0.900      91.6
0.950     180.0
0.970     241.0
0.990     450.0
0.999    1201.0
1.000    1439.0
Name: AMCH3A, dtype: float64

In [24]:
#exclude anything above 99th percentile, which in this case is 450 minutes
#amcheckin.AMCH3A[ amcheckin.AMCH3A > 450 ] = 'LOW DATA QUALITY'

In [25]:
amcheckin.AMCH3A.value_counts()

5.0       6259
10.0      5431
1.0       4373
15.0      4011
30.0      3686
20.0      2576
60.0      2491
2.0       2105
3.0       2024
61.0      1393
4.0       1364
6.0       1149
45.0      1116
120.0      988
90.0       822
8.0        666
121.0      642
25.0       627
7.0        609
40.0       530
12.0       436
180.0      400
9.0        344
35.0       340
181.0      296
75.0       287
11.0       280
150.0      267
14.0       235
13.0       227
          ... 
336.0        1
472.0        1
248.0        1
494.0        1
359.0        1
310.0        1
338.0        1
1380.0       1
366.0        1
437.0        1
468.0        1
790.0        1
231.0        1
382.0        1
233.0        1
690.0        1
208.0        1
411.0        1
182.0        1
652.0        1
224.0        1
990.0        1
413.0        1
203.0        1
433.0        1
407.0        1
378.0        1
555.0        1
324.0        1
263.0        1
Name: AMCH3A, Length: 428, dtype: int64

### AMCH5 Data Cleaning (Total Hours Slept)

In [26]:
amcheckin.AMCH5.describe()

count    54106.000000
mean       395.307230
std        126.233963
min          1.000000
25%        360.000000
50%        420.000000
75%        480.000000
max       1439.000000
Name: AMCH5, dtype: float64

In [27]:
amcheckin.AMCH5.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

0.010       1.0
0.050     120.0
0.500     420.0
0.600     430.0
0.800     481.0
0.850     495.0
0.900     522.0
0.950     545.0
0.970     580.0
0.990     630.0
0.999     781.0
1.000    1439.0
Name: AMCH5, dtype: float64

In [28]:
#exclude anything above 99th percentile 630 min (10.5 hrs, slightly more relaxed than normative data, 10h)
#amcheckin.AMCH5[ amcheckin.AMCH5 > 630 ] = 'LOW DATA QUALITY'

In [29]:
amcheckin.AMCH5.value_counts()

420.0     3531
480.0     3205
360.0     2658
421.0     2575
1.0       2227
450.0     2224
390.0     1980
361.0     1980
481.0     1816
300.0     1482
540.0     1224
510.0     1130
330.0     1105
301.0     1067
435.0      675
541.0      654
405.0      630
465.0      583
240.0      565
375.0      547
270.0      525
241.0      481
495.0      412
440.0      406
600.0      380
430.0      353
380.0      334
345.0      333
570.0      325
410.0      315
          ... 
21.0         1
641.0        1
669.0        1
23.0         1
795.0        1
132.0        1
860.0        1
136.0        1
825.0        1
790.0        1
129.0        1
76.0         1
924.0        1
16.0         1
673.0        1
153.0        1
701.0        1
70.0         1
69.0         1
1050.0       1
760.0        1
1387.0       1
42.0         1
189.0        1
1381.0       1
49.0         1
689.0        1
657.0        1
162.0        1
717.0        1
Name: AMCH5, Length: 660, dtype: int64

### Create Internal & External Copies of the Data

In [30]:
#download the data 
sharing_info = pd.read_excel(syn.get('syn21958546').path)
healthCodes_with_broadsharing = sharing_info[sharing_info.sharing == 'all_qualified_researchers']

EXTERNAL_AMCHECKIN_DATA = amcheckin[amcheckin.participantId.isin(healthCodes_with_broadsharing['participant id'])]
EXTERNAL_AMCHECKIN_DATA.shape

(49480, 9)

### Number of unique participants

In [31]:
len(EXTERNAL_AMCHECKIN_DATA.participantId.unique())

5266

### Total number of observations

In [32]:
len(EXTERNAL_AMCHECKIN_DATA)

49480

### External - Upload to Synapse

In [33]:
SH_EXTERNAL_PROJECT = 'syn18492837'
table_schema_external = Schema(name='AM Check-in',
                               columns=as_table_columns(EXTERNAL_AMCHECKIN_DATA),
                               parent=SH_EXTERNAL_PROJECT)
amcheckin_synTable_external = syn.store(Table(table_schema_external,EXTERNAL_AMCHECKIN_DATA))
amcheckin_synTable_external

### Internal - Upload to Synapse

In [34]:
SH_INTERNAL_PROJECT = 'syn7066726'
table_schema_internal = Schema(name='AM Check-in Internal',
                               columns=as_table_columns(amcheckin), 
                               parent=SH_INTERNAL_PROJECT)
amcheckin_synTable_internal = syn.store(Table(table_schema_internal,amcheckin))

### Set Provenance

In [35]:
activity=Activity(name= 'AM Check-in data curation', 
                  description='Process and convert raw data to table format', 
                  used=amcheckin_raw_id, 
                  executed='https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_AM_Checkin.ipynb')
#Internal 
syn.setProvenance(amcheckin_synTable_internal, activity)

#External
syn.setProvenance(amcheckin_synTable_external, activity)

{u'createdBy': u'3334346',
 u'createdOn': u'2020-07-20T18:51:30.791Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'56a653b5-33b7-4689-b58d-62c51f1e9bde',
 u'id': u'10282148',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-07-20T18:51:30.791Z',
 u'name': u'AM Check-in data curation',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7115672', u'targetVersionNumber': 928},
   u'wasExecuted': False},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_AM_Checkin.ipynb',
   u'url': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_AM_Checkin.ipynb',
   u'wasExecuted': True}]}

{u'createdBy': u'3334346',
 u'createdOn': u'2020-07-20T18:52:17.712Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'4602334c-94ce-4352-865c-8494d1dd4bcc',
 u'id': u'10282149',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-07-20T18:52:17.712Z',
 u'name': u'AM Check-in data curation',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7115672', u'targetVersionNumber': 928},
   u'wasExecuted': False},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_AM_Checkin.ipynb',
   u'url': u'https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_AM_Checkin.ipynb',
   u'wasExecuted': True}]}